# HSV Color Calibration Tool

GUI環境なしでHSVパラメータを調整するためのJupyterノートブック

## 機能
- インタラクティブなスライダーでHSV範囲を調整
- リアルタイムでマスクと検出結果をプレビュー
- 設定をYAMLファイルに保存

In [ ]:
# 必要なライブラリのインポート
import cv2
import numpy as np
import yaml
from pathlib import Path
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
import matplotlib.pyplot as plt

# matplotlibの設定
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 5]
plt.rcParams['figure.dpi'] = 100

In [ ]:
# 検出器のインポート（scriptsディレクトリから）
import sys
sys.path.append('/workspace/scripts')

try:
    from color_shape_detector import ColorShapeDetector
    print("✓ ColorShapeDetector loaded successfully")
except ImportError as e:
    print(f"✗ Failed to import ColorShapeDetector: {e}")
    print("  Make sure /workspace/scripts/color_shape_detector.py exists")

## 1. 画像の読み込み

調整したい画像のパスを指定してください。

In [ ]:
# === 画像パスを指定 ===
IMAGE_PATH = "/workspace/videos/sample.jpg"  # ← ここを変更

# 画像読み込み
image = cv2.imread(IMAGE_PATH)
if image is None:
    print(f"✗ Cannot read image: {IMAGE_PATH}")
    print("\nAvailable files in /workspace/videos/:")
    !ls -la /workspace/videos/
else:
    print(f"✓ Image loaded: {IMAGE_PATH}")
    print(f"  Size: {image.shape[1]}x{image.shape[0]}")
    
    # RGB変換して表示
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(image_rgb)
    plt.title("Original Image")
    plt.axis('off')
    plt.show()

## 2. HSVキャリブレーション

スライダーを動かしてHSV範囲を調整します。

In [ ]:
class HSVCalibratorJupyter:
    """Jupyter用HSVキャリブレーター"""
    
    # プリセット値
    PRESETS = {
        'blue': {'h_min': 95, 'h_max': 130, 's_min': 100, 's_max': 255, 'v_min': 100, 'v_max': 255},
        'yellow': {'h_min': 20, 'h_max': 45, 's_min': 100, 's_max': 255, 'v_min': 100, 'v_max': 255},
        'full': {'h_min': 0, 'h_max': 179, 's_min': 0, 's_max': 255, 'v_min': 0, 'v_max': 255},
    }
    
    def __init__(self, image):
        self.image = image
        self.image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.hsv = cv2.cvtColor(cv2.GaussianBlur(image, (5, 5), 0), cv2.COLOR_BGR2HSV)
        
        # 出力エリア
        self.output = widgets.Output()
        
        # スライダー作成
        self.sliders = {
            'h_min': widgets.IntSlider(value=0, min=0, max=179, description='H min'),
            'h_max': widgets.IntSlider(value=179, min=0, max=179, description='H max'),
            's_min': widgets.IntSlider(value=0, min=0, max=255, description='S min'),
            's_max': widgets.IntSlider(value=255, min=0, max=255, description='S max'),
            'v_min': widgets.IntSlider(value=0, min=0, max=255, description='V min'),
            'v_max': widgets.IntSlider(value=255, min=0, max=255, description='V max'),
        }
        
        # スライダーのスタイル設定
        for slider in self.sliders.values():
            slider.style.handle_color = '#2196F3'
            slider.layout.width = '400px'
            slider.observe(self._on_change, names='value')
        
        # プリセットボタン
        self.preset_buttons = widgets.HBox([
            widgets.Button(description='Blue Preset', button_style='info'),
            widgets.Button(description='Yellow Preset', button_style='warning'),
            widgets.Button(description='Reset', button_style='danger'),
        ])
        self.preset_buttons.children[0].on_click(lambda b: self._apply_preset('blue'))
        self.preset_buttons.children[1].on_click(lambda b: self._apply_preset('yellow'))
        self.preset_buttons.children[2].on_click(lambda b: self._apply_preset('full'))
        
        # 現在値表示ラベル
        self.info_label = widgets.HTML(value="")
    
    def _apply_preset(self, preset_name):
        """プリセットを適用"""
        preset = self.PRESETS[preset_name]
        for key, value in preset.items():
            self.sliders[key].value = value
    
    def _on_change(self, change):
        """スライダー変更時のコールバック"""
        self._update_display()
    
    def _get_current_values(self):
        """現在のスライダー値を取得"""
        return {k: v.value for k, v in self.sliders.items()}
    
    def _create_mask(self):
        """現在の設定でマスクを作成"""
        values = self._get_current_values()
        lower = np.array([values['h_min'], values['s_min'], values['v_min']])
        upper = np.array([values['h_max'], values['s_max'], values['v_max']])
        
        mask = cv2.inRange(self.hsv, lower, upper)
        
        # モルフォロジー変換
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        return mask
    
    def _update_display(self):
        """表示を更新"""
        with self.output:
            clear_output(wait=True)
            
            # マスク作成
            mask = self._create_mask()
            result = cv2.bitwise_and(self.image_rgb, self.image_rgb, mask=mask)
            
            # 輪郭検出
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # 結果画像に輪郭を描画
            result_with_contours = result.copy()
            detected_info = []
            
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 500:
                    # 輪郭近似
                    epsilon = 0.02 * cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, epsilon, True)
                    vertices = len(approx)
                    
                    # 凸包
                    hull = cv2.convexHull(contour)
                    
                    # バウンディングボックス
                    x, y, w, h = cv2.boundingRect(hull)
                    
                    # 描画
                    cv2.drawContours(result_with_contours, [hull], -1, (0, 255, 0), 2)
                    cv2.rectangle(result_with_contours, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    cv2.putText(result_with_contours, f"v={vertices}", (x, y-5),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                    
                    detected_info.append(f"Area: {int(area)}, Vertices: {vertices}, BBox: ({x},{y},{w},{h})")
            
            # 3列で表示
            fig, axes = plt.subplots(1, 3, figsize=(16, 5))
            
            axes[0].imshow(self.image_rgb)
            axes[0].set_title('Original')
            axes[0].axis('off')
            
            axes[1].imshow(mask, cmap='gray')
            axes[1].set_title('Mask')
            axes[1].axis('off')
            
            axes[2].imshow(result_with_contours)
            axes[2].set_title(f'Result ({len(detected_info)} objects)')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # 検出情報表示
            if detected_info:
                print("\n検出されたオブジェクト:")
                for i, info in enumerate(detected_info, 1):
                    print(f"  {i}. {info}")
        
        # 情報ラベル更新
        values = self._get_current_values()
        self.info_label.value = f"""
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; font-family: monospace;">
            <b>Current HSV Range:</b><br>
            H: [{values['h_min']}, {values['h_max']}]<br>
            S: [{values['s_min']}, {values['s_max']}]<br>
            V: [{values['v_min']}, {values['v_max']}]<br><br>
            <b>YAML format:</b><br>
            hsv_lower: [{values['h_min']}, {values['s_min']}, {values['v_min']}]<br>
            hsv_upper: [{values['h_max']}, {values['s_max']}, {values['v_max']}]
        </div>
        """
    
    def display(self):
        """UIを表示"""
        # スライダーをグループ化
        h_sliders = widgets.VBox([self.sliders['h_min'], self.sliders['h_max']], 
                                  layout=widgets.Layout(margin='0 20px 0 0'))
        s_sliders = widgets.VBox([self.sliders['s_min'], self.sliders['s_max']],
                                  layout=widgets.Layout(margin='0 20px 0 0'))
        v_sliders = widgets.VBox([self.sliders['v_min'], self.sliders['v_max']])
        
        slider_box = widgets.HBox([h_sliders, s_sliders, v_sliders])
        
        # 全体レイアウト
        ui = widgets.VBox([
            widgets.HTML("<h3>HSV Range Sliders</h3>"),
            slider_box,
            widgets.HTML("<br>"),
            self.preset_buttons,
            widgets.HTML("<br>"),
            self.info_label,
            widgets.HTML("<hr>"),
            self.output
        ])
        
        display(ui)
        self._update_display()
    
    def get_config(self, color_name='color'):
        """現在の設定を辞書で取得"""
        values = self._get_current_values()
        return {
            color_name: {
                'hsv_lower': [values['h_min'], values['s_min'], values['v_min']],
                'hsv_upper': [values['h_max'], values['s_max'], values['v_max']]
            }
        }
    
    def save_config(self, filepath, color_name='color'):
        """設定をYAMLファイルに保存"""
        config = self.get_config(color_name)
        with open(filepath, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        print(f"✓ Saved to: {filepath}")
        print(yaml.dump(config, default_flow_style=False))

In [ ]:
# キャリブレーターを起動
if image is not None:
    calibrator = HSVCalibratorJupyter(image)
    calibrator.display()
else:
    print("画像を読み込んでください")

## 3. 設定の保存

調整が完了したら、以下のセルで設定を保存できます。

In [ ]:
# === 青色の設定を保存 ===
# calibrator.save_config('/workspace/scripts/blue_config.yaml', 'blue')

In [ ]:
# === 黄色の設定を保存 ===
# calibrator.save_config('/workspace/scripts/yellow_config.yaml', 'yellow')

In [ ]:
# === 現在の設定を表示 ===
if 'calibrator' in dir():
    print("Current configuration:")
    print(yaml.dump(calibrator.get_config('color'), default_flow_style=False))

## 4. 検出テスト

`detector_config.yaml`を使って実際の検出をテストします。

In [ ]:
def test_detection(image_path, config_path='/workspace/scripts/detector_config.yaml'):
    """検出をテスト"""
    # 画像読み込み
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Cannot read: {image_path}")
        return
    
    # 検出器初期化
    detector = ColorShapeDetector(config_path)
    
    # 検出実行
    detections = detector.detect(frame)
    
    # 結果描画
    result = detector.draw_detections(frame, detections, draw_hull=True, draw_vertices=True)
    
    # マスク取得
    masks = detector.get_debug_masks(frame)
    
    # 表示
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('Original')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title(f'Detection Result ({len(detections)} objects)')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(masks['blue'], cmap='gray')
    axes[1, 0].set_title('Blue Mask')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(masks['yellow'], cmap='gray')
    axes[1, 1].set_title('Yellow Mask')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 検出結果の詳細
    print("\n検出結果:")
    print("=" * 60)
    for i, det in enumerate(detections, 1):
        print(f"{i}. {det.class_name}")
        print(f"   BBox: {det.bbox}")
        print(f"   Vertices: {det.vertices}")
        print(f"   Confidence: {det.confidence:.2f}")
    
    if not detections:
        print("検出なし")
    
    return detections

In [ ]:
# 検出テスト実行
detections = test_detection(IMAGE_PATH)

## 5. 動画からフレームを抽出してテスト

In [ ]:
def extract_and_test_frame(video_path, frame_number=0, config_path='/workspace/scripts/detector_config.yaml'):
    """動画から指定フレームを抽出してテスト"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open: {video_path}")
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames: {total_frames}")
    
    # 指定フレームに移動
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"Cannot read frame {frame_number}")
        return
    
    print(f"Frame {frame_number} extracted")
    
    # 一時ファイルに保存
    temp_path = '/tmp/temp_frame.jpg'
    cv2.imwrite(temp_path, frame)
    
    # テスト実行
    return test_detection(temp_path, config_path)

In [ ]:
# 動画からフレーム抽出してテスト（必要に応じてコメント解除）
# VIDEO_PATH = "/workspace/videos/input.mp4"
# extract_and_test_frame(VIDEO_PATH, frame_number=100)

## 6. 設定ファイルの編集

現在の`detector_config.yaml`を確認・編集できます。

In [ ]:
# 現在の設定を表示
CONFIG_PATH = '/workspace/scripts/detector_config.yaml'

try:
    with open(CONFIG_PATH, 'r') as f:
        current_config = yaml.safe_load(f)
    print("Current detector_config.yaml:")
    print("=" * 40)
    print(yaml.dump(current_config, default_flow_style=False))
except FileNotFoundError:
    print(f"Config file not found: {CONFIG_PATH}")

In [ ]:
def update_config(color, hsv_lower, hsv_upper, config_path='/workspace/scripts/detector_config.yaml'):
    """設定ファイルのHSV範囲を更新"""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    if color in config:
        config[color]['hsv_lower'] = hsv_lower
        config[color]['hsv_upper'] = hsv_upper
        
        with open(config_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        
        print(f"✓ Updated {color} HSV range:")
        print(f"  hsv_lower: {hsv_lower}")
        print(f"  hsv_upper: {hsv_upper}")
    else:
        print(f"✗ Color '{color}' not found in config")

# 使用例（必要に応じてコメント解除）：
# update_config('blue', [95, 100, 100], [130, 255, 255])
# update_config('yellow', [20, 100, 100], [45, 255, 255])

---

## Tips

### 青いシール（冷蔵）の調整
- H: 95-130 (青の範囲)
- S: 100-255 (彩度が高い)
- V: 100-255 (明度が高い)

### 黄色いシール（冷凍）の調整
- H: 20-45 (黄色の範囲)
- S: 100-255 (彩度が高い)
- V: 100-255 (明度が高い)

### 検出がうまくいかない場合
1. S（彩度）のmin値を下げてみる
2. V（明度）のmin値を下げてみる
3. H（色相）の範囲を広げてみる

### 誤検出が多い場合
1. S, Vのmin値を上げる
2. Hの範囲を狭める
3. `min_area`を大きくする